In [1]:
#Note: Can add synthesis params (like use_dsp48) as 3rd series params

In [2]:
#HRR_mode: noHR only (1), noHR or SoE_HRR (2), SoE_HRR only (3), OTI (4)
#N start: 1, 1, 2, 2
#1: no HRR and NOEPS, 2 and 3: NOEPS given, HRR calculated, 4: HRR given, NOEPS calculated

In [3]:
import sys
import math

import import_ipynb
from REGISTERED_Generator import REGISTERED_Gen

from MatrixMatrixMultiply_OTI_V1_1 import MatrixMatrixMultiply_OTI
from MatrixMatrixMultiply_SoE_V1_1 import MatrixMatrixMultiply_SoE
from MatrixVectorMultiply_SoE_V2_5 import MatrixVectorMultiply_SoE

from OTItoSeries_V1_00 import OTItoSeries

importing Jupyter notebook from REGISTERED_Generator.ipynb
importing Jupyter notebook from MatrixMatrixMultiply_OTI_V1_1.ipynb
importing Jupyter notebook from OTItoSeries_V1_00.ipynb
importing Jupyter notebook from MatrixMatrixMultiply_SoE_V1_1.ipynb
importing Jupyter notebook from MatrixVectorMultiply_SoE_V2_5.ipynb


In [4]:
# # re_import after changes
# del sys.modules['REGISTERED_Generator']
# from REGISTERED_Generator import REGISTERED_Gen

In [5]:
# # re_import after changes
# del sys.modules['OTItoSeries_V1_00']
# from OTItoSeries_V1_00 import OTItoSeries

In [6]:
# # re_import after changes
# del sys.modules['MatrixMatrixMultiply_OTI_V1_1']
# from MatrixMatrixMultiply_OTI_V1_1 import MatrixMatrixMultiply_OTI

In [7]:
def CEIL(a,b):
    return (a+b-1)//b

In [8]:
#read-only globals: NoOfParameters, HRRmode, Print_To_File, Files_Location
#writed globals: parameters_result
def generate_params(i):
    global NoOfRresults, NoOfRresultsT, NOEPS
    if i==NoOfParameters:
        NoOfRresults += 1
        NoOfRresultsT += make_files()
        return
#     print(i,parameters[i])
    param = parameters[i]
    vals = parameter_values[param]
    start = vals[0]
    end = vals[1]
    if param == "N":
        for val in range(start,end+1):
            parameters_result[param] = val
            generate_params(i+1)
    elif param in ["M","Q"]:
        for val in range(start,end+1):
            parameters_result[param] = val
            generate_params(i+1)
    elif param in ["ENABLE_INPUT_LATCH","ENABLE_COLUMN_LATCH","ENABLE_ROW_LATCH","ENABLE_OTI_OUTPUT_REGISTERS"]:
        for val in range(start,end+1):
            parameters_result[param] = val
            generate_params(i+1)
    elif param == "PR":
        for val in range(start,min(end,parameters_result['M'])+1):
            parameters_result["PR"] = val
            generate_params(i+1)
    elif param == "PC":
        for val in range(start,min(end,parameters_result['Q'])+1):
            parameters_result["PC"] = val
            generate_params(i+1)
    elif param == "HRR":
        for val in range(start,min(end,parameters_result['N'])+1):
            parameters_result["HRR"] = val
            generate_params(i+1)
    elif param == "NOEPS":
        if HRRmode == 2:
            endH = parameters_result["N"]
        elif HRRmode == 3:
            endH = parameters_result["N"]-1
        for val in range(start,min(end,endH)+1):
                parameters_result["NOEPS"] = val
                generate_params(i+1)
    elif param == "MAMCS":
        if HRRmode == 1:
            NOEPS = parameters_result["N"]
        elif HRRmode in [2,3]:
            NOEPS = parameters_result["NOEPS"]
        elif HRRmode == 4:
            NOEPS = CEIL(parameters_result["N"],parameters_result["HRR"])
        for val in range(start,min(end,NOEPS)+1):
            parameters_result["MAMCS"] = val
            generate_params(i+1)
    elif param == "ADDER_SIZE":
        if parameters_result["MAMCS"] == NOEPS:
            parameters_result["ADDER_SIZE"] = 0
            generate_params(i+1)
        else: #MAMCS < NOEPS
            for val in range(start,min(end,CEIL(NOEPS,parameters_result["MAMCS"]))+1):
                parameters_result["ADDER_SIZE"] = val
                generate_params(i+1)

In [9]:
def make_files():
    global top, Current_Files_Location, NoOfVResults, modules
    print(parameters_result)
    modules = {}
    top = function(parameters_result, Print_To_File, Files_Location, 1, modules)
    print(modules)
    print(top)
    Current_Files_Location = Files_Location+top+"/"
    REGISTERED_Gen(top, Print_To_File, Current_Files_Location)
    make_prj()
    make_tcl()
    NoOfVResults = 0
    NOVR = generate_verilog_params_and_syn_files(0)
    print("NoOfVResults = {}".format(NOVR))
    print()
    return NOVR

In [10]:
def make_prj():
    with open(Current_Files_Location+top+"_REGISTERED.prj","w") as prj_file:
        print("verilog work \""+top+"_REGISTERED.v\"", file=prj_file)
        for module in modules:
            print("verilog work \""+module+".v\"", file=prj_file)

In [11]:
def make_tcl():
    if Print_To_File<=0:
        tcl_file=sys.stdout
    else:
        tcl_file=open(Current_Files_Location+top+".tcl", 'w+')
        
    print("set myProject \""+top+"\"\n", file=tcl_file)
    
    print("proc set_project_props {} {", file=tcl_file)
    print("\tproject set family \""+family+"\"", file=tcl_file)
    print("\tproject set device \""+device+"\"", file=tcl_file)
    print("\tproject set package \""+package+"\"", file=tcl_file)
    print("\tproject set speed \""+speed_grade+"\"\n}\n", file=tcl_file)

    print("proc add_source_files {} {", file=tcl_file)
    print("\txfile add \""+top+"_REGISTERED.v\"", file=tcl_file)
    for module in modules:
        print("\txfile add \""+module+".v\"", file=tcl_file)
    print("\t#Set the Top Module", file=tcl_file)
    print("\tproject set top \""+top+"_REGISTERED\"\n}\n", file=tcl_file)
    
    print("""set proj_exts [ list ise xise gise ]
foreach ext $proj_exts {
   set proj_name "${myProject}.$ext"
   if { [ file exists $proj_name ] } { 
      file delete $proj_name
   }
}

project new $myProject
set_project_props
add_source_files
project close""", file=tcl_file)

    if Print_To_File>0:
        tcl_file.close()
    
    with open(Current_Files_Location+"CreateProject.bat","w") as CPB_file:
        print("xtclsh "+top+".tcl 2> TCLinfo.log", file=CPB_file)

In [12]:
def generate_verilog_params_and_syn_files(i):
    global NoOfVResults
    if i==NoOfVeilogParameters:
        NoOfVResults += 1
        #print(verilog_parameters_result)
        #make diffferent synthesis settings
        make_SYN()
        make_TMPR()
        return
    param = verilog_parameters[i]
    vals = verilog_parameter_values[param]
    start = vals[0]
    end = vals[1]
    if param in ["IN_WIDTH","INPUT_REG_DEPTH","MULT_PIPE_DEPTH"]:
        for val in range(start,end+1):
            verilog_parameters_result[param] = val
            generate_verilog_params_and_syn_files(i+1)
    return NoOfVResults

In [13]:
#current layout: spartan 6
def make_SYN():
    global VP_name_append, STMPR_file_name
    if Print_To_File<=0:
        xst_file=sys.stdout
    else:
        STMPR_file_name = top+"_REGISTERED"
        VP_name_append = ""
        for param in verilog_parameters:
            VP_name_append += "_{}".format(verilog_parameters_result[param])
        STMPR_file_name = STMPR_file_name+VP_name_append
        xst_file=open(Current_Files_Location+STMPR_file_name+".xst", 'w+')
        
    print("""set -tmpdir "./"
set -xsthdpdir "xst"
run""", file=xst_file)
    print("-ifn "+top+"_REGISTERED.prj", file=xst_file)
    print("-ofn "+top+"_REGISTERED", end='', file=xst_file)
    for param in verilog_parameters:
        print("_{}".format(verilog_parameters_result[param]), end='', file=xst_file)
    print("\n-ofmt NGC", file=xst_file)
    print("-p "+device+speed_grade+"-"+package, file=xst_file)
    print("-top "+top+"_REGISTERED", file=xst_file)
    print("""-opt_mode Speed
-opt_level 1
-power NO
-iuc NO
-keep_hierarchy No
-netlist_hierarchy As_Optimized
-rtlview Yes
-glob_opt AllClockNets
-read_cores YES
-write_timing_constraints NO
-cross_clock_analysis NO
-hierarchy_separator /
-bus_delimiter <>
-case Maintain
-slice_utilization_ratio 100
-bram_utilization_ratio 100
-dsp_utilization_ratio 100
-lc Auto
-reduce_control_sets Auto
-fsm_extract YES -fsm_encoding Auto
-safe_implementation No
-fsm_style LUT
-ram_extract Yes
-ram_style Auto
-rom_extract Yes
-shreg_extract YES
-rom_style Auto
-auto_bram_packing NO
-resource_sharing YES
-async_to_sync NO
-shreg_min_size 2
-use_dsp48 Auto
-iobuf YES
-max_fanout 100000
-bufg 16
-register_duplication YES
-register_balancing No
-optimize_primitives NO
-use_clock_enable Auto
-use_sync_set Auto
-use_sync_reset Auto
-iob Auto
-equivalent_register_removal YES
-slice_utilization_ratio_maxmargin 5""", file=xst_file)
    print("-generics {", end='', file=xst_file)
    for param in verilog_parameters:
        print(param+"={} ".format(verilog_parameters_result[param]), end='', file=xst_file)
    print("}", end='', file=xst_file)

    if Print_To_File>0:
        xst_file.close()
    
    with open(Current_Files_Location+"Synthesize"+VP_name_append+".bat","w") as SYNB_file:
        print("xst -intstyle silent -ifn "+STMPR_file_name+".xst -ofn "+STMPR_file_name+".syr", file=SYNB_file)

In [14]:
def make_TMPR():
    with open(Current_Files_Location+"TMPR"+VP_name_append+".bat","w") as TMPRB_file:
        #print("ngdbuild -intstyle silent -quiet -dd _ngo -nt timestamp -i -p "+device+speed_grade+"-"+package+" "+STMPR_file_name+".ngc "+STMPR_file_name+".ngd", file=TMPRB_file)
        print("ngdbuild -dd _ngo -nt timestamp -i -p "+device+speed_grade+"-"+package+" "+STMPR_file_name+".ngc "+STMPR_file_name+".ngd", file=TMPRB_file)
        print("map -intstyle silent -p "+device+speed_grade+"-"+package+" -w -logic_opt off -ol high -t 1 -xt 0 -register_duplication off -r 4 -global_opt off -mt off -ir off -pr off -lc off -power off -o "+STMPR_file_name+"_map.ncd "+STMPR_file_name+".ngd "+STMPR_file_name+".pcf", file=TMPRB_file)
        print("par -w -intstyle silent -ol high -mt off "+STMPR_file_name+"_map.ncd "+STMPR_file_name+".ncd "+STMPR_file_name+".pcf", file=TMPRB_file)
        print("trce -intstyle silent -v 3 -s 3 -n 3 -fastpaths -xml "+STMPR_file_name+".twx "+STMPR_file_name+".ncd -o "+STMPR_file_name+".twr "+STMPR_file_name+".pcf", file=TMPRB_file)

In [15]:
functions_list = []
functions_names_list = []
parameters_list = []
HRR_mode_list = []

verilog_parameters_list = []

Functions Start

In [16]:
#MatrixMatrixMultiply_OTI

functions_list.append(MatrixMatrixMultiply_OTI)
functions_names_list.append("MatrixMatrixMultiply_OTI")

parameters_list.append(["M","N","Q","PR","PC","HRR","MAMCS","ADDER_SIZE",
                       "ENABLE_INPUT_LATCH","ENABLE_COLUMN_LATCH","ENABLE_ROW_LATCH","ENABLE_OTI_OUTPUT_REGISTERS"])

HRR_mode_list.append(4)

verilog_parameters_list.append(["IN_WIDTH", "INPUT_REG_DEPTH", "MULT_PIPE_DEPTH"])

In [17]:
#MatrixMatrixMultiply_SoE

functions_list.append(MatrixMatrixMultiply_SoE)
functions_names_list.append("MatrixMatrixMultiply_SoE")

parameters_list.append(["M","N","Q","PR","PC","NOEPS","MAMCS","ADDER_SIZE",
                       "ENABLE_INPUT_LATCH","ENABLE_COLUMN_LATCH","ENABLE_ROW_LATCH","ENABLE_OTI_OUTPUT_REGISTERS"])

HRR_mode_list.append(3)

verilog_parameters_list.append(["IN_WIDTH", "INPUT_REG_DEPTH", "MULT_PIPE_DEPTH"])

In [18]:
#MatrixVectorMultiply_SoE

In [19]:
# #M_Type2

# functions_list.append(M_Type2)
# functions_names_list.append("M_Type2")

# HRR_mode_list.append(2)

In [20]:
#OTItoSeries

functions_list.append(OTItoSeries)
functions_names_list.append("OTItoSeries")

parameters_list.append(["N","HRR","ENABLE_INPUT_LATCH","ENABLE_OTI_OUTPUT_REGISTERS"])

HRR_mode_list.append(4)

verilog_parameters_list.append(["IN_WIDTH"])

Functions End

In [21]:
print("choose function:")
i=0;
for n in functions_names_list:
    i+=1
    print("{}: ".format(i)+n)
index = int(input("Enter function number([1,{}]):".format(i)))-1

choose function:
1: MatrixMatrixMultiply_OTI
2: MatrixMatrixMultiply_SoE
3: OTItoSeries
Enter function number([1,3]):1


In [22]:
notation = { #will be updated after getting values
    "ENABLE_INPUT_LATCH": "[0(No),1(Yes)]",
    "ENABLE_COLUMN_LATCH": "[0(No),1(Yes)]",
    "ENABLE_ROW_LATCH": "[0(No),1(Yes)]",
    "ENABLE_OTI_OUTPUT_REGISTERS": "[0(No),1(Yes)]",
    #
    "M": ">=1",
    "Q": ">=1"
}

In [23]:
function = functions_list[index]
parameters = parameters_list[index]
HRRmode = HRR_mode_list[index]
if HRRmode in [1,2]:
    notation["N"] = ">=1"
elif HRRmode in [3,4]:
    notation["N"] = ">=2"

In [24]:
# parameter_values = {}
# notValid = False
# for parameter in parameters:
#     print(parameter+": "+notation[parameter])
#     startI = int(input(parameter+" start:"))
#     endI = int(input(parameter+" end:"))
#     if parameter == "N":
#         end = endI
#         if HRRmode in [1,2]:
#             start = max(startI,1)
#             notation["NOEPS"] = "[1,{}]".format(end) #2
#         elif HRRmode in [3,4]:
#             start = max(startI,2)
#             notation["HRR"] = "[2,{}]".format(end) #4
#             notation["NOEPS"] = "[1,{}]".format(end-1) #3
#         if HRRmode == 1:
#             notation["MAMCS"] = "[1,{}]".format(end)
#     elif parameter == "M":
#         start = max(startI,1)
#         end = endI
#         notation["PR"] = "[{},{}]".format(start,end)
#     elif parameter == "Q":
#         start = max(startI,1)
#         end = endI
#         notation["PC"] = "[{},{}]".format(start,end)
#     elif parameter in ["ENABLE_INPUT_LATCH","ENABLE_COLUMN_LATCH","ENABLE_ROW_LATCH","ENABLE_OTI_OUTPUT_REGISTERS"]: #Yes,No params
#         start = max(startI,0)
#         end = min(endI,1)
#     elif parameter == "PR":
#         start = max(startI,1)
#         end = min(endI,parameter_values["M"][1])
#     elif parameter == "PC":
#         start = max(startI,1)
#         end = min(endI,parameter_values["Q"][1])
#     elif parameter == "HRR": #4
#         start = max(startI,2)
#         end = min(endI,parameter_values["N"][1])
#         notation["MAMCS"] = "[1,{}]".format(CEIL(parameter_values["N"][1],start))
#     elif parameter == "NOEPS":
#         start = max(startI,1)
#         if HRRmode == 2:
#             end = min(endI,parameter_values["N"][1])
#         elif HRRmode == 3:
#             end = min(endI,parameter_values["N"][1]-1)
#         notation["MAMCS"] = "[1,{}]".format(end)
#     elif parameter == "MAMCS":
#         start = max(startI,1)
#         if HRRmode == 1:
#             NOEPSmaxC = parameter_values["N"][1]
#         elif HRRmode in [2,3]:
#             NOEPSmaxC = parameter_values["NOEPS"][1]
#         elif HRRmode == 4:
#             NOEPSmaxC = CEIL(parameter_values["N"][1],parameter_values["HRR"][0])
#         end = min(endI,NOEPSmaxC)
#         NOEPSmax = end
#         if start == NOEPSmax:
#             notation["ADDER_SIZE"] = "==0"      
#         else: #MAMCS min < NOEPSmax
#             notation["ADDER_SIZE"] = "[2,{}]".format(CEIL(NOEPSmax,start))
#     elif parameter == "ADDER_SIZE":
#         if parameter_values["MAMCS"][0] == NOEPSmax:
#             start = max(startI,0)
#             end = min(endI,0)
#         else: #MAMCS min < NOEPSmax
#             start = max(startI,2)
#             end = min(endI,CEIL(NOEPSmax,parameter_values["MAMCS"][0]))
#     else:
#         print("unknown parameter! will break.")
#         notValid = True
#         break;
#     print("calculated start: {}".format(start))
#     print("calculated end: {}".format(end))
#     if end<start:
#         print("Invalid value range! will break.")
#         notValid = True
#         break;
#     parameter_values[parameter]=[start,end]

In [25]:
parameter_values = {'MAMCS': [1, 1], 'PC': [1, 1], 'ENABLE_INPUT_LATCH': [0, 1], 'Q': [1, 1], 'HRR': [2, 2], 'ADDER_SIZE': [0, 0], 'M': [1, 1], 'ENABLE_OTI_OUTPUT_REGISTERS': [1, 1], 'PR': [1, 1], 'ENABLE_COLUMN_LATCH': [1, 1], 'N': [2, 2], 'ENABLE_ROW_LATCH': [1, 1]}

#temp changes to values
# parameter_values["NOEPS"][0] = 1 #2
# parameter_values["NOEPS"][1] = 4
# print(parameter_values)

print(parameter_values)

{'ENABLE_COLUMN_LATCH': [1, 1], 'ENABLE_ROW_LATCH': [1, 1], 'N': [2, 2], 'M': [1, 1], 'Q': [1, 1], 'ADDER_SIZE': [0, 0], 'MAMCS': [1, 1], 'HRR': [2, 2], 'PC': [1, 1], 'ENABLE_OTI_OUTPUT_REGISTERS': [1, 1], 'ENABLE_INPUT_LATCH': [0, 1], 'PR': [1, 1]}


In [26]:
INPUT_REG_DEPTH_max_valid = 2
MULT_PIPE_DEPTH_max_valid = 2

verilog_notation = {
    "IN_WIDTH": ">=1",
    "INPUT_REG_DEPTH": "[0,{}]".format(INPUT_REG_DEPTH_max_valid),
    "MULT_PIPE_DEPTH": "[0,{}]".format(MULT_PIPE_DEPTH_max_valid)
}

In [27]:
verilog_parameters = verilog_parameters_list[index]

In [28]:
# verilog_parameter_values = {}
# VnotValid = False
# for parameter in verilog_parameters:
#     print(parameter+": "+verilog_notation[parameter])
#     startI = int(input(parameter+" start:"))
#     endI = int(input(parameter+" end:"))
#     if parameter == "IN_WIDTH":
#         start = max(startI,1)
#         end = endI
#     elif parameter == "INPUT_REG_DEPTH":
#         start = max(startI,0)
#         end = min(endI,INPUT_REG_DEPTH_max_valid)
#     elif parameter == "MULT_PIPE_DEPTH":
#         start = max(startI,0)
#         end = min(endI,MULT_PIPE_DEPTH_max_valid)
#     else:
#         print("unknown parameter! will break.")
#         VnotValid = True
#         break;
#     print("calculated start: {}".format(start))
#     print("calculated end: {}".format(end))
#     if end<start:
#         print("Invalid value range! will break.")
#         VnotValid = True
#         break;
#     verilog_parameter_values[parameter]=[start,end]

In [29]:
verilog_parameter_values = {'INPUT_REG_DEPTH': [0, 0], 'IN_WIDTH': [10, 11], 'MULT_PIPE_DEPTH': [0, 0]}

print(verilog_parameter_values)

{'IN_WIDTH': [10, 11], 'MULT_PIPE_DEPTH': [0, 0], 'INPUT_REG_DEPTH': [0, 0]}


In [30]:
print("parameter count: {}".format(len(parameters)))
print()

Print_To_File = 1

Files_Location = "D:/Thesis_results/"

family = "Spartan6"
device = "xc6slx150"
package = "fgg900"
speed_grade = "-3"

NoOfRresults = 0
NoOfRresultsT = 0
NoOfParameters = len(parameters)
parameters_result = {}
#modules = {}

NoOfVResults = 0
NoOfVeilogParameters = len(verilog_parameters)
verilog_parameters_result = {}

generate_params(0)

print("no of results: {}".format(NoOfRresults))
print("no of results T: {}".format(NoOfRresultsT))

parameter count: 12

{'ENABLE_COLUMN_LATCH': 1, 'N': 2, 'ENABLE_ROW_LATCH': 1, 'Q': 1, 'ADDER_SIZE': 0, 'MAMCS': 1, 'HRR': 2, 'PC': 1, 'ENABLE_OTI_OUTPUT_REGISTERS': 1, 'PR': 1, 'ENABLE_INPUT_LATCH': 0, 'M': 1}
{'MatrixMatrixMultiply_1_2_1_1PR_1PC_OTIR_HRx2_S1E_C1_A0_NIL_CL_RL': 'MatrixMatrixMultiply_OTI', 'MatrixMatrixMultiply_1_2_1_1PR_1PC_S1E_HRx2_C1_A0_CL_RL': 'MatrixMatrixMultiply_SoE', 'OTItoSeries_2_HRx2_S1E_OR_NIL': 'OTItoSeries', 'MatrixVectorMultiply_1_2_1PR_S1E_HRx2_C1_A0_VL': 'MatrixVectorMultiply_SoE'}
MatrixMatrixMultiply_1_2_1_1PR_1PC_OTIR_HRx2_S1E_C1_A0_NIL_CL_RL
NoOfVResults = 2

{'ENABLE_COLUMN_LATCH': 1, 'NOEPS': 1, 'N': 2, 'ENABLE_ROW_LATCH': 1, 'Q': 1, 'ADDER_SIZE': 0, 'MAMCS': 1, 'HRR': 2, 'PC': 1, 'ENABLE_OTI_OUTPUT_REGISTERS': 1, 'PR': 1, 'ENABLE_INPUT_LATCH': 1, 'M': 1}
{'OTItoSeries_2_HRx2_S1E_OR_IL': 'OTItoSeries', 'MatrixMatrixMultiply_1_2_1_1PR_1PC_S1E_HRx2_C1_A0_CL_RL': 'MatrixMatrixMultiply_SoE', 'MatrixVectorMultiply_1_2_1PR_S1E_HRx2_C1_A0_VL': 'MatrixVe